# SAP Developer Challenge – APIs
This notebook contains my solutions for the [SAP Developer Challenge – APIs](https://blogs.sap.com/2023/08/01/sap-developer-challenge-apis/).

In [ ]:
import requests

COMMUNITY_ID = "ceedee666"
NW_BASE_URL = "https://developer-challenge.cfapps.eu10.hana.ondemand.com/odata/v4/northbreeze"

## Task 0 - Learn to share your task results
The task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-0-learn-to-share-your-task/).

In [ ]:
def get_hash_for_value(value, headers={"CommunityID" : COMMUNITY_ID}):
    url = f"https://developer-challenge.cfapps.eu10.hana.ondemand.com/v1/hash(value='{value}')"
    r = requests.get(url, headers=headers)
    return r.status_code, r.text

print(get_hash_for_value("this-is-the-year-of-the-api"))

## Task 1 - List the Northwind entity sets
The task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-1-list-the-northwind-entity/).

In [ ]:
def northwind_entity_sets():
    northwind_service_url = "https://services.odata.org/V4/Northwind/Northwind.svc/"
    r = requests.get(northwind_service_url)
    
    entity_sets = [ v["name"] for v in r.json()["value"] if v["kind"] == "EntitySet"]
    
    return sorted(entity_sets)

entity_sets = northwind_entity_sets()
print(f"The Northwind service provides {len(entity_sets)} entity sets.")

value = ",".join(entity_sets)
print("The entity sets provided by the Northwind service are:", value)
print("The resulting hash value is:", get_hash_for_value(value)[1])

## Task 2 - Calculate Northbreeze product stock 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-2-calculate-northbreeze/). For this task the we need to
- Calculate the sum of the `UnitsInStock` for all products
- Only products that are not discontinued (ie. `Discontinued: false`) should be counted.

I solved this task using three approaches:
1. Getting all data form the service and perform the filtering and aggregation in Python 🐍
2. Filtering the data using OData '$filter` and the aggregation in Python 🐍
3. Using OData data for filtering and aggregation as described [here](https://github.com/qmacro/odata-v4-and-cap/blob/main/slides.md#data-aggregation)


In [ ]:
PRODCUTS_URL = NW_BASE_URL + "/Products"

def product_stock_in_python():
    r = requests.get(PRODCUTS_URL)
    products = r.json()["value"]
    products_in_stock = {p["ProductID"] : p["UnitsInStock"] for p in products if p["Discontinued"] == False}
    return products_in_stock

def product_stock_with_odata_filter():
    r = requests.get(PRODCUTS_URL + "?$filter=Discontinued eq false&$select=ProductID,UnitsInStock")
    products = r.json()["value"]
    return {p["ProductID"] : p["UnitsInStock"] for p in products}

def product_stock_with_odata():
    r = requests.get(PRODCUTS_URL + "?$apply=filter(Discontinued eq false)/aggregate(UnitsInStock with sum as TotalStock)")
    return r.json()["value"][0]["TotalStock"]




print("Solution 1: Performating filtering and aggreation in Python:")
products_in_stock = product_stock_in_python()
print(f"The service returns {len(products_in_stock.keys())} that are not disontinued. The total stock of these products is {sum(products_in_stock.values())}")

print("\nSolution 2: Performating filtering in OData and aggreation in Python:")
products_in_stock = product_stock_with_odata_filter()
print(f"The service returns {len(products_in_stock.keys())} that are not disontinued. The total stock of these products is {sum(products_in_stock.values())}")

print("\nSolution 3: Performating filtering and aggregation in OData:")
total_stock = product_stock_with_odata()
print(f"The total stock of these products is {total_stock}")


print("\nThe resulting hash value is:", get_hash_for_value(total_stock)[1])

## Task 3 - Have a Northbreeze product selected for you 
The detailed task description is available [here](https://groups.community.sap.com/t5/application-development/sap-developer-challenge-apis-task-3-have-a-northbreeze-product/td-p/277972). For this task the we need to use the `selectProduct` action to let the service select a product based on our community id.

In [13]:
import urllib.parse
SELECT_PRODCUT_URL = NW_BASE_URL + "/selectProduct"

def call_select_product():
    payload = {"communityid" : COMMUNITY_ID}
    r = requests.post(SELECT_PRODCUT_URL, json=payload)
    return r.json()["value"]

selected_product = call_select_product()
print(f"The Northbreeze service selected the product {selected_product} for my community ID")

encoded_product = urllib.parse.quote_plus(selected_product)
print(f"The URL-encoded product is: {encoded_product}")

print("\nThe resulting hash value is:", get_hash_for_value(encoded_product)[1])
    

The Northbreeze service selected the product Röd Kaviar for my community ID
The URL-encoded product is: R%C3%B6d+Kaviar

The resulting hash value is: 802f380c0012c393c38e60b6f3a2cd67717b441eb7bb8d27f2d5de2bfebbb9e8
